# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [2]:
url = 'https://scanner.tradingview.com/america/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change|1","change|5","change|15","change|60","change|240","change","change|1W","change|1M","Perf.3M","Volatility.W","average_volume_90d_calc","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"name","sortOrder":"asc"},"range":[0,8033]}
data = requests.post(url, json = payload)

In [3]:
stocks_data = data.json()['data']
len(stocks_data)

7275

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
stocks_data[0]

{'s': 'NYSE:A',
 'd': ['agilent-technologies',
  'A',
  144.475,
  0.03114726,
  -0.06914673,
  -0.00691898,
  -0.20715371,
  0.42387603,
  0.37865629,
  0.37865629,
  1.76445728,
  -5.04436411,
  2.31808442,
  1305275.58888889,
  'Agilent Technologies, Inc.',
  'stock',
  'common',
  'delayed_streaming_900',
  100,
  1,
  'false',
  0,
  'USD',
  'USD']}

In [5]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][14]
    stock['Price'] = data['d'][2]
    stock['1min CHG'] = data['d'][3]
    stock['5min CHG'] = data['d'][4]
    stock['15min CHG'] = data['d'][5]
    stock['1Hr CHG'] = data['d'][6]
    stock['4Hr CHG'] = data['d'][7]
    stock['1Day CHG'] = data['d'][8]
    stock['1Week CHG'] = data['d'][9]
    stock['1Month CHG'] = data['d'][10]
    stock['3Month CHG'] = data['d'][11]
    stock['Volatility'] = data['d'][12]
    stock['AvgVolume'] = data['d'][13]
    
    stats_from_tradingview.append(stock)

In [6]:
stats_from_tradingview[0]['5min CHG']

-0.06914673

In [7]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("stocks_all_performance_short_term.json", "w") as outfile:
    outfile.write(json_object)

In [8]:
hqm_dataframe = pd.read_json('stocks_all_performance_short_term.json')
available_stocks = pd.read_json('curated_stocks_all.json')
available_stocks_list = available_stocks['symbol'].tolist()

In [9]:
hqm_dataframe = hqm_dataframe[hqm_dataframe['Symbol'].isin(available_stocks_list)]

In [10]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",144.4750,0.031147,-0.069147,-0.006919,-0.207154,0.423876,0.378656,0.378656,1.764457,-5.044364,2.318084,1.305276e+06
1,AA,Alcoa Corporation,54.3401,0.055259,0.018413,0.018413,-0.730994,-2.161383,-2.125180,-2.125180,11.034123,14.883932,4.587341,5.149864e+06
10,AAMC,Altisource Asset Management Corp Com,53.5500,-0.093284,0.000000,-0.204994,0.657706,0.657706,0.657706,0.657706,2.960969,142.307692,5.421809,8.905078e+03
11,AAME,Atlantic American Corporation,2.3500,1.240738,1.240738,1.240738,1.240738,-1.669526,-1.669526,-1.669526,-6.000000,-12.962963,4.328924,4.121633e+03
12,AAN,"Aarons Holdings Company, Inc.",12.5100,0.080000,0.320770,0.562701,-0.950119,-2.189210,-2.112676,-2.112676,-12.822300,0.968523,5.674934,3.652194e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7267,ZUO,"Zuora, Inc.",10.1100,-0.049456,-0.049456,-0.148221,0.647410,0.949051,0.898204,0.898204,19.362456,36.991870,6.318154,1.148706e+06
7268,ZVIA,Zevia PBC,3.6600,-0.543478,-0.543478,-1.081081,-5.321158,-2.400000,-2.400000,-2.400000,4.871060,-15.473441,8.686966,1.334354e+05
7272,ZYME,Zymeworks Inc.,8.0300,0.000000,0.124688,0.124688,0.249688,0.249688,0.249688,0.249688,-2.311436,5.104712,5.201201,1.082280e+06
7273,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4800,0.000000,0.000000,0.000000,0.418200,0.051902,-3.264813,-3.264813,3.896104,-22.580645,8.200430,3.140892e+05


In [11]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
mean_val = hqm_dataframe['1Week CHG'].mean()
hqm_dataframe['1Week CHG'].fillna(value=mean_val, inplace=True)
hqm_dataframe.isna().any()

Symbol        False
Name          False
Price         False
1min CHG      False
5min CHG      False
15min CHG     False
1Hr CHG       False
4Hr CHG       False
1Day CHG      False
1Week CHG     False
1Month CHG    False
3Month CHG    False
Volatility    False
AvgVolume     False
dtype: bool

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [12]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [13]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume
0,A,"Agilent Technologies, Inc.",144.4750,0.031147,-0.069147,-0.006919,-0.207154,0.423876,0.378656,0.378656,1.764457,-5.044364,2.318084,1.305276e+06
1,AA,Alcoa Corporation,54.3401,0.055259,0.018413,0.018413,-0.730994,-2.161383,-2.125180,-2.125180,11.034123,14.883932,4.587341,5.149864e+06
2,AAMC,Altisource Asset Management Corp Com,53.5500,-0.093284,0.000000,-0.204994,0.657706,0.657706,0.657706,0.657706,2.960969,142.307692,5.421809,8.905078e+03
3,AAME,Atlantic American Corporation,2.3500,1.240738,1.240738,1.240738,1.240738,-1.669526,-1.669526,-1.669526,-6.000000,-12.962963,4.328924,4.121633e+03
4,AAN,"Aarons Holdings Company, Inc.",12.5100,0.080000,0.320770,0.562701,-0.950119,-2.189210,-2.112676,-2.112676,-12.822300,0.968523,5.674934,3.652194e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,ZUO,"Zuora, Inc.",10.1100,-0.049456,-0.049456,-0.148221,0.647410,0.949051,0.898204,0.898204,19.362456,36.991870,6.318154,1.148706e+06
3114,ZVIA,Zevia PBC,3.6600,-0.543478,-0.543478,-1.081081,-5.321158,-2.400000,-2.400000,-2.400000,4.871060,-15.473441,8.686966,1.334354e+05
3115,ZYME,Zymeworks Inc.,8.0300,0.000000,0.124688,0.124688,0.249688,0.249688,0.249688,0.249688,-2.311436,5.104712,5.201201,1.082280e+06
3116,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4800,0.000000,0.000000,0.000000,0.418200,0.051902,-3.264813,-3.264813,3.896104,-22.580645,8.200430,3.140892e+05


In [14]:
list(hqm_dataframe.columns.values)

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume']

In [15]:
time_periods = [
#     '1min CHG',
#     '5min CHG',
#     '15min CHG',
#     '1Hr CHG',
#     '4Hr CHG',
#     '1Day CHG',
    '1Week CHG',
    '1Month CHG',
    '3Month CHG',
#     'Volatility'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume,1Week CHG Percentile,1Month CHG Percentile,3Month CHG Percentile
0,A,"Agilent Technologies, Inc.",144.4750,0.031147,-0.069147,-0.006919,-0.207154,0.423876,0.378656,0.378656,1.764457,-5.044364,2.318084,1.305276e+06,67.864015,58.915972,31.494548
1,AA,Alcoa Corporation,54.3401,0.055259,0.018413,0.018413,-0.730994,-2.161383,-2.125180,-2.125180,11.034123,14.883932,4.587341,5.149864e+06,15.651058,95.798589,74.567030
2,AAMC,Altisource Asset Management Corp Com,53.5500,-0.093284,0.000000,-0.204994,0.657706,0.657706,0.657706,0.657706,2.960969,142.307692,5.421809,8.905078e+03,74.246312,70.397691,99.262348
3,AAME,Atlantic American Corporation,2.3500,1.240738,1.240738,1.240738,1.240738,-1.669526,-1.669526,-1.669526,-6.000000,-12.962963,4.328924,4.121633e+03,21.135343,7.344452,17.350866
4,AAN,"Aarons Holdings Company, Inc.",12.5100,0.080000,0.320770,0.562701,-0.950119,-2.189210,-2.112676,-2.112676,-12.822300,0.968523,5.674934,3.652194e+05,15.779346,2.212957,46.343810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,ZUO,"Zuora, Inc.",10.1100,-0.049456,-0.049456,-0.148221,0.647410,0.949051,0.898204,0.898204,19.362456,36.991870,6.318154,1.148706e+06,78.383579,98.749198,91.597178
3114,ZVIA,Zevia PBC,3.6600,-0.543478,-0.543478,-1.081081,-5.321158,-2.400000,-2.400000,-2.400000,4.871060,-15.473441,8.686966,1.334354e+05,13.181527,81.719051,14.464400
3115,ZYME,Zymeworks Inc.,8.0300,0.000000,0.124688,0.124688,0.249688,0.249688,0.249688,0.249688,-2.311436,5.104712,5.201201,1.082280e+06,64.207826,17.126363,56.029506
3116,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4800,0.000000,0.000000,0.000000,0.418200,0.051902,-3.264813,-3.264813,3.896104,-22.580645,8.200430,3.140892e+05,7.408595,76.651700,9.332906


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [16]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [17]:
hqm_dataframe

,Symbol,Name,Price,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,Volatility,AvgVolume,1Week CHG Percentile,1Month CHG Percentile,3Month CHG Percentile,HQM Score
0,A,"Agilent Technologies, Inc.",144.4750,0.031147,-0.069147,-0.006919,-0.207154,0.423876,0.378656,0.378656,1.764457,-5.044364,2.318084,1.305276e+06,67.864015,58.915972,31.494548,52.758178
1,AA,Alcoa Corporation,54.3401,0.055259,0.018413,0.018413,-0.730994,-2.161383,-2.125180,-2.125180,11.034123,14.883932,4.587341,5.149864e+06,15.651058,95.798589,74.567030,62.005559
2,AAMC,Altisource Asset Management Corp Com,53.5500,-0.093284,0.000000,-0.204994,0.657706,0.657706,0.657706,0.657706,2.960969,142.307692,5.421809,8.905078e+03,74.246312,70.397691,99.262348,81.302117
3,AAME,Atlantic American Corporation,2.3500,1.240738,1.240738,1.240738,1.240738,-1.669526,-1.669526,-1.669526,-6.000000,-12.962963,4.328924,4.121633e+03,21.135343,7.344452,17.350866,15.276887
4,AAN,"Aarons Holdings Company, Inc.",12.5100,0.080000,0.320770,0.562701,-0.950119,-2.189210,-2.112676,-2.112676,-12.822300,0.968523,5.674934,3.652194e+05,15.779346,2.212957,46.343810,21.445371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,ZUO,"Zuora, Inc.",10.1100,-0.049456,-0.049456,-0.148221,0.647410,0.949051,0.898204,0.898204,19.362456,36.991870,6.318154,1.148706e+06,78.383579,98.749198,91.597178,89.576652
3114,ZVIA,Zevia PBC,3.6600,-0.543478,-0.543478,-1.081081,-5.321158,-2.400000,-2.400000,-2.400000,4.871060,-15.473441,8.686966,1.334354e+05,13.181527,81.719051,14.464400,36.454993
3115,ZYME,Zymeworks Inc.,8.0300,0.000000,0.124688,0.124688,0.249688,0.249688,0.249688,0.249688,-2.311436,5.104712,5.201201,1.082280e+06,64.207826,17.126363,56.029506,45.787898
3116,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4800,0.000000,0.000000,0.000000,0.418200,0.051902,-3.264813,-3.264813,3.896104,-22.580645,8.200430,3.140892e+05,7.408595,76.651700,9.332906,31.131067


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [18]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [19]:
cols = list(hqm_dataframe.columns.values)
cols

['Symbol',
 'Name',
 'Price',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
 'Volatility',
 'AvgVolume',
 '1Week CHG Percentile',
 '1Month CHG Percentile',
 '3Month CHG Percentile',
 'HQM Score']

In [21]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[['Symbol',
 'Name',
 'Price',
 'HQM Score',
 'AvgVolume',
 'Volatility',
 '1min CHG',
 '5min CHG',
 '15min CHG',
 '1Hr CHG',
 '4Hr CHG',
 '1Day CHG',
 '1Week CHG',
 '1Month CHG',
 '3Month CHG',
#  '1min CHG Percentile',
#  '5min CHG Percentile',
#  '15min CHG Percentile',
#  '1Hr CHG Percentile',
#  '4Hr CHG Percentile',
#  '1Day CHG Percentile',
 '1Week CHG Percentile',
 '1Month CHG Percentile',
 '3Month CHG Percentile',
#  'Volatility Percentile'
]]

In [22]:
hqm_dataframe[:50]

,Symbol,Name,Price,HQM Score,AvgVolume,Volatility,1min CHG,5min CHG,15min CHG,1Hr CHG,4Hr CHG,1Day CHG,1Week CHG,1Month CHG,3Month CHG,1Week CHG Percentile,1Month CHG Percentile,3Month CHG Percentile
0,TRKA,"Troika Media Group, Inc.",0.7200,99.978619,2.917996e+07,25.922801,0.718411,3.818789,5.957321,10.835529,40.580884,39.805825,39.805825,83.439490,415.021459,99.967928,99.967928,100.000000
1,KALA,"Kala Pharmaceuticals, Inc.",19.7400,99.946547,1.888341e+06,25.568295,-0.101926,-1.259072,-2.136115,-6.584157,19.249395,17.850746,17.850746,94.866732,297.183099,99.903784,100.000000,99.935856
2,BBAI,"BigBear.ai, Inc.",3.5696,99.454779,5.233097e+06,17.719640,0.484180,-0.565475,0.552113,-3.273358,12.605678,12.251572,12.251572,21.414966,264.244898,99.518922,98.973701,99.871713
3,BBIO,"BridgeBio Pharma, Inc.",17.1300,99.390635,1.585441e+06,6.882330,0.000000,0.411387,-1.326413,-1.731614,57.261029,57.589696,57.589696,50.000000,87.008734,100.000000,99.807569,98.364336
4,MSGM,Motorsport Games Inc.,10.9667,99.187513,1.016331e+06,13.694559,1.670512,-0.977798,1.362727,5.686256,17.677045,14.474948,14.474948,20.513187,112.162894,99.743425,98.813342,99.005773
5,EHTH,"eHealth, Inc.",9.4500,98.353645,8.627161e+05,15.223545,0.000000,0.000000,-0.211193,-1.664932,4.419890,4.651163,4.651163,28.396739,107.692308,96.568313,99.518922,98.973701
6,SUP,"Superior Industries International, Inc.",7.2500,98.300192,7.810768e+04,7.558602,-0.405777,-0.405777,0.702364,-0.024854,8.228700,7.566766,7.566766,33.272059,56.250000,98.909557,99.647210,96.343810
7,DUOL,"Duolingo, Inc.",124.3600,97.968783,4.262422e+05,8.176336,0.040277,-0.285038,-0.432855,-0.600209,4.072739,4.267628,4.267628,36.975438,81.407086,96.183451,99.743425,97.979474
8,PBPB,Potbelly Corporation,8.4342,97.722899,6.919523e+04,5.503461,0.407143,0.526818,1.616867,2.981685,6.358134,6.224181,6.224181,17.141667,59.738636,98.075690,98.332264,96.760744
9,ALGN,"Align Technology, Inc.",352.0500,97.701518,1.128533e+06,4.420228,0.083853,0.234859,-0.598498,0.204906,5.239561,5.227762,5.227762,13.747981,88.503962,97.209750,97.434253,98.460552
